# Image Features

## Goals:
- learn feature extraction, first step of use image features for applications
- learn what characteristics make good image features
- learn different algorithms used to extract features in images

## 1. Feature Dectecion
- features are points of interest in an image
- points of interest should have the following characteristics:
    - Saliency: distinctive, identifiable, and different from its imemediate neighborhood
    - Reppeatability: can be found in multiple images using same operations
    - Locality: occupies a relatively small subset of image space
        - should not change much under small changes in image
    - Quantity: enough points represented in the image
    - Efficency: reasonable computing time as a preprocessing step


Extraction:

- repetitive texture less patches are challenging to detect consistently
- patches with large contrast changes (gradients) are easier to detect (edges)
- gradients in at least two (significantly) different orientations are the easiest to detect (corners)
    - famous corner algorithm: Harris Corner Detection


### 1.1 Feature Detection Algorithms
- Harris -> corners
  - easy to compute, but not sacale invariant
- Harris-Laplace -> corners
  - scale invariant
- Features from acclerated segment test (FAST) -> corners
  - machine learning approach for fast corner detection
- Laplacian of Gaussian (LOG) detector -> blobs
  - scale invariant, but not rotation invariant
- Difference of Gaussian (DOG) detector -> blobs
  - approximate LOG, but faster


### 1.1 Harris Corner Detection


## 2. Feature Descriptors

**Goals:**
- what characteristics make a good feature descriptor
- different algorithms used to extract feature descriptors from images


**Features/Descriptors**
- Feature: points of interest in an image defined by pixel coordinates $[u,v]$
- Descriptor: an N-dimensional vector that provides a summary of the image information around the detected feature $\{f_1, f_2, ..., f_N\}$

- Feature descriptors should have the following characteristics:
    - Distinctiveness: different features should have different descriptors
    - Robustness: descriptors should be able to be matched even if the image is transformed
    - Compactness: descriptors should be small enough to be stored and compared efficiently
    - Efficiency: descriptors should be able to be computed quickly


**How to describe a feature**
- obsolute intensity values of pixels in a local neighborhood: 
    - not robust to illumination changes
- relative intensity (i.e., gradient) values of pixels in a local neighborhood:
    - feature is invariant to obsolute intensity values
    - feature is senstive to deformations
- color histogram of pixels in a local neighborhood:
    - invariant to changes in scale and rotation
    - sensitive to spatial layout
- spatial histograms
    - sensitive to rotation
- orientation normalization
    - use the dominat image gradient direction to normalize the orientation of the path
        - save the orientation angle along with pixel intensity

**Multi-scale Oriented Patches (MOPS)**

**Histogram of Textons Descriptor**

**Histogram of oritend gradients (HOG)**

**SIFT: Scale Invariant Feature Transform**
- SIFT describes both a detector and descriptor
    - detection
        - multi-scale extrema detection: Gaussian pyramid + DOG
        - keypoint localization
        - orientation assignment
        - returns: $(x,y) \rightarrow\ location, $\sigma$ $\rightarrow$ scale, $\theta$ $\rightarrow$ orientation

    - keypoint descriptor


- SIFT is used in many state-of-art systems
- Combined with DOG feature detector, SIFT descriptors provide a scalue, rotation, and illumation invariant feature detector/descriptor pair


**Other Feature Descriptors**:
- Speeded Up Robust Features (SURF)
- Gradient Location and Orientation Histogram (GLOH)
- Binary Robust Independent Elementary Features (BRIEF)
- Oriented FAST and Rotated BRIEF (ORB)


## 3. Feature Matching

given a feature and its descriptor in image 1, find the best matching in image 2.


**Distance Function**

- SSD: square sum of differences
- SAD: sum of absolute differences
- Hamming distance: for binary descriptors

**Brute Force Matching**
- define a distance function $d(f_i, f_j)$ that compares two descriptors
- for every feature $f_i$ in image 1:
    - compute distance $d(f_i, f_j)$ for every feature $f_j$ in image 2
    - find the `cloest` match $f_c$ that has the smallest distance to $f_i$

Problem of the above approach is:
- if $f_i$ in image 1 doesn't have a match in image 2, then the `closest` match will be returned as the $f_j$ with the smallest distance, even the distance is pretty big.
- solution is to use a threshold to filter out the bad matches

**Nearest Neighbor Matching**
- define a distance function $d(f_i, f_j)$ that compares two descriptors
- define a distance threshold $\delta$
- for every feature $f_i$ in image 1:
    - compute distance $d(f_i, f_j)$ for every feature $f_j$ in image 2
    - find the `closest` match $f_c$ that has the smallest distance to $f_i$
    - if $d(f_i, f_c) < \delta$, then $f_c$ is a good match for $f_i$, and keep it


- Brute force matching might be fast enough for large amounts of features
- Use KD-tree or other data structures to speed up nearest neighbor search


### Ambiguous Matches

- if there are multiple matches for a feature, then the feature is ambiguous
- ambiguous matches can be resolved by:
    - using a better feature detector/descriptor
    - using a better distance function
    - using a better matching algorithm
    - using a better threshold
    - using a better data structure for nearest neighbor search

**Distance Ratio**
- compute the distance $d(f_i,f_j)$ for each feature $f_i$ in image 1, and featuer $f_j$ in image 2.
- find the cloest match $f_c$
- find the second closest match $f_s$
- find how better the closest match is than the second closest match: $r = \frac{d(f_i,f_c)}{d(f_i,f_s)}$


**Brute Force Matching with Distance Ratio**
- define a distance function $d(f_i, f_j)$ that compares two descriptors
- define a distance ratio threshold $\rho$
- for every feature $f_i$ in image 1:
    - compute distance $d(f_i, f_j)$ for every feature $f_j$ in image 2
    - find the `closest` match $f_c$ and the second cloest match $f_s$
    - compute their distance ratio $r$
    - keep matches with $r < \rho$


## 4. Outlier Rejection

- outliers are wrong feature matching outputs, that can occur due to errors in any of the three stages of feature detection, description, and matching
- RANSAC is a popular algorithm for outlier rejection


**Why removing outliers**

The purpose of doing feature matching is to find the same feature in two different images, so that we can estimate the transformation between the two images.
The transformation of the images can then be used to estimate the camera pose. If there are outliers in the feature matching, then the estimated transformation will be wrong, and the estimated camera pose will be wrong too.

**RANSAC**

## 5. Visual Odemetry

**Visual Odemetry (VO)**
- the process of estimating the motion of a camera by analyzing the associated image sequences

pros:
- not affected by wheel slip in uneven terrain, rainy/snowy weather, or other adverse condition
- more accurate trajectory estimates compared to wheel odemetry

cons:
- need an external sensor to estimate absolute scale. scale cannot be estimated by a single sensor
- camera is a passive sensor, might not be robust to weather/illumination changes
- any form of ordemetry (incremetal state estimation) drifts over time


**VO Pipeline**
- feature detection
- feature matching
- outlier rejection
- motion estimation
- scale estimation
- loop closure
- map optimization

**VO vs SLAM**
- VO: only estimate the camera motion
- SLAM: estimate the camera motion and the map of the environment

**VO Problem**

given two frames of an image sequence $I_{k-1}, I_k$, estimate the camera motion $T_k$ between the two frames.

$$
T_k = \begin{bmatrix} R_{k,k-1} & t_{k,k-1} \\ 0 & 1 \end{bmatrix}
$$

where $R_k$ is the rotation matrix, and $t_k$ is the translation vector.

Concatenating these single movements allows the recovery of the full trajectory of the cammera


Pipeline for solving:
    - Given: $I_{k-1}, I_k$
    - Extract and match features $f_{k-1}$ and $f_k$
    - Estimate motion $T_k$ from $f_{k-1}$ and $f_k$

**Motion Estimation**
depends on feature representation:
- 2D to 2D: both $f_{k-1}$ and $f_k$ are defiend in image coordinates
    - for tracking object in image frame
    - visual tracking and image stabalization
- 3D to 3D: both $f_{k-1}$ and $f_k$ are defiend in 3D 
    - stereo camera or depth information
    - locate image features in 3-D space
- 3D to 2D: $f_{k-1}$ is defiend in 3D, and $f_k$ is their corresponding projection defiend in image coordinates

**3D-2D Motion Estimation**

problem:
- 3D world coordinates of features in frame k-1
- 2D image coordinates of features in frame k

camera projection model:

$$
\begin{bmatrix} su \\ sv \\ s \end{bmatrix} = \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} R & t \end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}
$$

camera instrinsic matrix $K$ is known from camera calibration, world coordinates and image coordinates are also known.

unknowns:
- rotation matrix $R$ 
- translation vector $t$

![motion-estimation](./resources/img/motion-estimation.png)

### PNP Perspective-n-Point

PnP:
- solve for initial guess of $R$ and $t$ using Direct Linear Transform (DLT)
    - form a linear model and solve for $R$ and $t$ using SVD
- improve solution using Levemberg-Marquardt (LM) optimization
- need at least 3 points to solve P3P, 4 if we dont want ambiguous solutions. 

